In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
pd_ads=pd.read_csv('D:\\sale\\t_ads.csv')
pd_comment=pd.read_csv('D:\\sale\\t_comment.csv')
pd_order=pd.read_csv('D:\\sale\\t_order.csv')
pd_product=pd.read_csv('D:\\sale\\t_product.csv')
pd_sale=pd.read_csv('D:\\sale\\t_sales_sum.csv')

E:\python-ana\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
sale_1036=pd_sale[pd_sale['shop_id']==1036]
comment_1036=pd_comment[pd_comment['shop_id']==1036]
order_1036=pd_order[pd_order['shop_id']==1036]
product_1036=pd_product[pd_product['shop_id']==1036]
ads_1036=pd_ads[pd_ads['shop_id']==1036]

sale_1036['dt']=pd.to_datetime(sale_1036['dt'])
comment_1036['create_dt']=pd.to_datetime(comment_1036['create_dt'])
order_1036['ord_dt']=pd.to_datetime(order_1036['ord_dt'])
product_1036['on_dt']=pd.to_datetime(product_1036['on_dt'])
ads_1036['create_dt']=pd.to_datetime(ads_1036['create_dt'])

sale_1036=sale_1036.sort_values(['dt'],ascending=True)
comment_1036=comment_1036.sort_values(['create_dt'],ascending=True)
order_1036=order_1036.sort_values(['ord_dt'],ascending=True)
product_1036=product_1036.sort_values(['on_dt'],ascending=True)
ads_1036=ads_1036.sort_values(['create_dt'],ascending=True)

sale_1036.to_csv('D:\\sale\\sale_1036.csv')
comment_1036.to_csv('D:\\sale\\comment_1036.csv')
order_1036.to_csv('D:\\sale\\order_1036.csv')
product_1036.to_csv('D:\\sale\\product_1036.csv')
ads_1036.to_csv('D:\\sale\\ads_1036.csv')

E:\python-ana\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
E:\python-ana\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\python-ana\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
order_1036.drop('pid',axis=1,inplace=True)
order_1036=order_1036.groupby(['ord_dt']).sum()

comment_1036.drop('shop_id',axis=1,inplace=True)
order_1036.drop('shop_id',axis=1,inplace=True)
ads_1036.drop('shop_id',axis=1,inplace=True)


#order_1036.set_index("ord_dt", inplace=True) order_1036已将ord_dt作为索引
sale_1036.set_index("dt", inplace=True)
comment_1036.set_index("create_dt", inplace=True)
product_1036.set_index("on_dt", inplace=True)
ads_1036.set_index("create_dt", inplace=True)

ads_1036

,charge,consume
create_dt,,
2016-08-19,143472.98,0.00
2016-08-20,0.00,2563.01
2016-08-21,0.00,7138.29
2016-08-22,0.00,9110.21
2016-08-23,0.00,13637.82
2016-08-24,0.00,15265.27
2016-08-25,0.00,11760.37
2016-08-26,0.00,8048.08
2016-08-27,0.00,1125.64


In [5]:
#date_list=pd.date_range('2016-6-1','2017-4-30')
date=pd.DataFrame(index=pd.date_range('2016-8-1','2017-4-30'))
data=pd.concat([date,comment_1036, order_1036,ads_1036], axis=1, join_axes=[date.index])
data.to_csv('D:\\sale\\data_1036.csv')
data

,bad_num,cmmt_num,dis_num,good_num,mid_num,sale_amt,offer_amt,offer_cnt,rtn_cnt,rtn_amt,ord_cnt,user_cnt,charge,consume
2016-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-03,NaN,NaN,NaN,NaN,NaN,6462.10,2369.94,81.0,2.0,58.12,124.0,124.0,NaN,NaN
2016-08-04,NaN,NaN,NaN,NaN,NaN,5605.59,2290.11,74.0,0.0,0.00,107.0,107.0,NaN,NaN
2016-08-05,0.0,144.0,0.0,144.0,0.0,5328.07,2242.29,72.0,0.0,0.00,104.0,103.0,NaN,NaN
2016-08-06,0.0,198.0,2.0,196.0,2.0,5957.45,2372.53,77.0,0.0,0.00,118.0,118.0,NaN,NaN
2016-08-07,0.0,240.0,0.0,240.0,0.0,5793.08,2289.76,76.0,0.0,0.00,112.0,112.0,NaN,NaN
2016-08-08,0.0,266.0,0.0,266.0,0.0,5626.25,2320.65,75.0,0.0,0.00,109.0,109.0,NaN,NaN
2016-08-09,0.0,360.0,0.0,360.0,0.0,5372.68,2218.75,72.0,0.0,0.00,104.0,104.0,NaN,NaN
2016-08-10,0.0,304.0,0.0,304.0,0.0,5462.70,2204.41,71.0,0.0,0.00,102.0,102.0,NaN,NaN
